In [1]:
import os
import numpy as np
from subprocess import call
import sys
sys.path.insert(0, './src/')
from src import util

experimental_conditions = [750, 500, 250]
data_dir = "/vol/tensusers3/camghane/ASR/LibriSpeech_train_clean_360/data/train-clean-360/normalized"


copy_subsets = False
merge_files = True

## Copy Subsets of LibriSpeech 360

In [2]:
def cp_dir(source, target):
    call(['cp', '-a', source, target]) # Linux

In [3]:
if copy_subsets:
    speakers = np.array(os.listdir(data_dir))
    for nr_speakers in experimental_conditions:

        new_data_dir = "/vol/tensusers3/camghane/ASR/jdy/data/trainset{}/normalized".format(nr_speakers)

        if not os.path.exists(new_data_dir):
            os.makedirs(new_data_dir)

        speakers = speakers[np.random.choice(len(speakers), size=nr_speakers, replace=False)]

        for speaker_id in speakers:
            source_loc = data_dir + "/{}".format(speaker_id)
            target_loc = new_data_dir + "/{}".format(speaker_id)
            cp_dir(source_loc, target_loc)


## Merging Files for each Dataset

In [4]:
if merge_files:
    max_nr_of_speakers = 10

    for nr_speakers in experimental_conditions:
        data_dir = "/vol/tensusers3/camghane/ASR/jdy/data/trainset{}/normalized/".format(nr_speakers)
        new_dir = "/vol/tensusers3/camghane/ASR/jdy/data/trainset{}/merged/".format(nr_speakers)
        if not os.path.exists(new_dir):
            os.makedirs(new_dir)
        util.merge_audiofiles(data_dir, new_dir, max_nr_of_speakers)

/home/jnaus/NumberOfSpeakerEstimation/src/util.py:193: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  files_per_speaker = np.array(files_per_speaker)
0it [00:00, ?it/s]

106703 files found in total
Now starting to merge files...
5126
8498
1913
1290
8575
6060
7498
6763
7665
7120
2581
4331
5588
3972
2853
6788
1629
534
868
1387
671
5538
5189
7868
1987
5606
6763
7994
6388
5157
5656
101
1800
4071
2582
6119
1365
3482
2254
188
4057
7789
1448
6937
4519
8459
6308
2618
3083
166
159
288
7730
464
3340
6575
8758
534
6157
2532
8266
2769
6620
2229
4734


0it [00:00, ?it/s]

7832
274
3989
3001
157
2827
698
1387
2853
5802
4356
4425
6167
4133
7383
7754
6139
2769
2673
2812
55
4800
6080
6286
1060
2348
8490
2085
2751
1121
2254
6981
114
3361
1337
5538
1923
6494
4356
7294
815
8163
6918
3537
7011
2812
1849
835
231
2674
6538
3972
8687
3630
716
1851
639
6544
525
4519
7837
8421
957
2573
3009
3493


0it [00:00, ?it/s]

...

0it [00:02, ?it/s]

5126
512
5039
7540
5337
2404
2532
716
8050
5746
7525
8066
5123
2146
3046
6339
1165
1487
1885
5133
5604
6038
4110
2618
5588
6160
7011
7825
7995
6499
3221
6544
3446
126
836
3989
6341
7959
119
7090
7962
434
166
5519
2012
868
7538
2146
5139
8776
8388
5655
1222
815
1913
188
8300
6782
3728
698
501
7991
5513
4425
7286
4860
7316
5304
7752
8527
8699
1112
868
1473
1806
8183
4856
2230
1224
459
2790
476
434
612
439
17
3092
398
3979


0it [00:03, ?it/s]

724
7956
6373
7416
1323
258
525
3972
7949
14
1456
373
1806
246
1264
2696
664
1417
5012
3638
8776
6406
5985
3540
1487
22
1379
2628
639
3866
3092
4770
3118
3330
3274
101


KeyboardInterrupt: 

## Delete Files per Dataset, for equally sized datasets

In [ ]:
for nr_speakers in experimental_conditionsental_conditions:
    data_dir = "/vol/tensusers3/camghane/ASR/jdy/data/trainset{}/merged/train/".format(nr_speakers)
    
    